#Experiment - degrade knowledge of e and i params and see if we can still pick up K35.

In [1]:
#can we find K34 while assuming basic values for the e and i parameters?
import copy
import numpy as np
import emcee
import corner
import matplotlib.pyplot as p
import astropy.units as u
import funcs

%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [75]:
# Kepler 35
cb = funcs.CBSystem(m1 = 0.8877,f1 = 1.,m2 = 0.8094,f2 = 0.3941,
                    ab = 0.17617,r1 = 1.0284 * u.Rsun.to('au'),r2 = 0.7861 * u.Rsun.to('au'),
                    eb = 0.1421,ib = np.deg2rad(90.4238),wb = 1.507,
                    fb = 0.06543,
                    mp = 0.127 * u.Mjup.to('Msun'),ap = 0.60347,rp = 0.728 * u.Rjupiter.to('au'),ep = 0.042,
                    ip = np.deg2rad(90.76),wp = 1.1541,fp = 1.3069,
                    Wp = np.deg2rad(-1.24),
                    t0 = 2454965.8500)
ab0 = cb.ab
mb0 = cb.m1 + cb.m2
p_p0 = (cb.ap**3/(cb.m1+cb.m2))**(1./2.)*365.25 #in days
fp0 = 1.3069
ip0 = np.deg2rad(90.76)
print(1/np.sqrt( (cb.m1+cb.m2)/cb.ab**3 )*365.25)
print(p_p0)

timing_precision = 30./86400./365.25 * (2*np.pi)  #in years/2pi
dt = 0.02
t = np.arange(2454953,2456424,dt)


20.7316772617
131.43772562486816


In [76]:
# make the reference stacked light curve set

#tts = funcs.reb_cb_c(cb,tmin=np.min(t),tmax=np.max(t))
tts = funcs.reb_cb(cb,tmin=np.min(t),tmax=np.max(t))
tts_2,tds_2 = funcs.reb_cb_dvm(cb,0,2,tmin=np.min(t),tmax=np.max(t), timing_precision=timing_precision)
    
ok = tts[:,0] == 20
#ref_tts = tts[ok]
ref_tts = tts_2[tds_2>0]
ref_tds = tds_2[tds_2>0]
print(ref_tts)

[ 2455332.87041014  2455460.2174625   2455586.42187743  2455712.356026
  2455838.3220782   2455964.64963706  2456092.50592814]


In [77]:
#degrading our knowledge (assume we know eb and wb):
#for now, radii left as we know them - this will change transit durations, and whether some marginal transits happen.

cb.ib = np.pi/2.
cb.ip = np.pi/2.
cb.Wp = 0.0
cb.ep = 0.0
#cb.wp left as is because it defines f_p
cb.mp = 0.0



In [78]:
#now scan across m2/m1, m1+m2, p_p, f_p, as if we were searching.
#mratset = np.linspace(cb.m2/cb.m1*0.9,cb.m2/cb.m1*1.1,21)
#mtotset = np.linspace(cb.m2+cb.m1*0.5,cb.m1+cb.m2*1.5,5)
mratset = [cb.m2/cb.m1]
mtotset = [cb.m2+cb.m1]

ppset = np.linspace(p_p0*0.97,p_p0*1.03,101)
fpset = np.linspace(cb.fp*0.98,cb.fp*1.02,11)
ipset = np.linspace(np.deg2rad(89.0),np.deg2rad(91.0),41)
dt = 0.02
t = np.arange(2454953,2456424,dt)

print(mtotset)
print(mratset)

[1.6971]
[0.9117945251774248]


In [79]:
tts_all = {}
tds_all = {}

#horrible for loop filling in a 7 fold dict of dicts
for mrat in mratset:
    print(mrat)
    if mrat not in tts_all.keys():
        tts_all[mrat] = {}
        tds_all[mrat] = {}
        
    for mtot in mtotset:
        cb.m2 = mtot / (1 + 1./mrat)
        cb.m1 = mtot / (1 + mrat)
        cb.ab = ab0 * ( (cb.m1+cb.m2)/mb0 )**(1/3.) #force pb to be fixed
        
        if mtot not in tts_all[mrat].keys():
            tts_all[mrat][mtot] = {}
            tds_all[mrat][mtot] = {}
            
        for pp in ppset:
            cb.ap = ( mtot * (pp/365.25)**2 )**(1./3.)
  
            if pp not in tts_all[mrat][mtot].keys():
                tts_all[mrat][mtot][str(pp)[:6]] = {}
                tds_all[mrat][mtot][str(pp)[:6]] = {}  
                
            for fp in fpset:
                cb.fp = fp
                if fp not in tts_all[mrat][mtot][str(pp)[:6]].keys():
                    tts_all[mrat][mtot][str(pp)[:6]][str(fp)[:6]] = {}
                    tds_all[mrat][mtot][str(pp)[:6]][str(fp)[:6]] = {}  
                    
                for ip in ipset:
                    cb.ip = ip
                    ttimes, tdurs = funcs.reb_cb_dvm(cb, 0, 2, tmin=np.min(t), 
                                                tmax=np.max(t), timing_precision=timing_precision)
                            
                    tts_all[mrat][mtot][str(pp)[:6]][str(fp)[:6]][str(ip)[:6]] = ttimes[tdurs>0]
                    tds_all[mrat][mtot][str(pp)[:6]][str(fp)[:6]][str(ip)[:6]] = tdurs[tdurs>0]

0.9117945251774248


KeyboardInterrupt: 

In [72]:
#make statistic from transit times and durs

timediffs = np.zeros([len(mratset),len(mtotset),len(ppset),len(fpset),len(ipset)])
durdiffs = np.zeros([len(mratset),len(mtotset),len(ppset),len(fpset),len(ipset)])
losttransits = np.zeros([len(mratset),len(mtotset),len(ppset),len(fpset),len(ipset)])
overlap = np.zeros([len(mratset),len(mtotset),len(ppset),len(fpset),len(ipset)])
                   
#horrible for loop
for imr,mrat in enumerate(mratset):
    for imt,mtot in enumerate(mtotset):
        for ipp,pp in enumerate(ppset):
            for ifp,fp in enumerate(fpset):
                for iip,ip in enumerate(ipset):
                
                    #check against ref_tts
                    #save difference, in tts and tds, across all transits. 
                    #save number of transits lost (or gained)
                    diff = 0
                    ddiff = 0
                    tlost = 0
                    for ref_tt,ref_td in zip(ref_tts,ref_tds):
                        tts = tts_all[mrat][mtot][str(pp)[:6]][str(fp)[:6]][str(ip)[:6]]
                        tds = tds_all[mrat][mtot][str(pp)[:6]][str(fp)[:6]][str(ip)[:6]]
                        if len(tts)>0:
                            if np.min(np.abs(tts - ref_tt)) < pp/4.: #then we have a transit
                                tidx = np.argmin(np.abs(tts - ref_tt))
                                diff += np.abs(ref_tt - tts[tidx])
                                ddiff += np.abs(ref_td - tds[tidx])
                                olap = np.min([ref_tt+ref_td/2.,tts[tidx]+tds[tidx]/2.]) - np.max([ref_tt-ref_td/2.,tts[tidx]-tds[tidx]/2.])
                                if olap < 0: olap = 0
                                #calculate fraction of overlap with reference
                                overlap[imr,imt,ipp,ifp,iip] += olap  
                            else:
                                tlost +=1
                        else:
                            tlost = len(ref_tts)
                            overlap[imr,imt,ipp,ifp,iip] = 0 
                    if (len(ref_tts)-tlost) > 0:
                        timediffs[imr,imt,ipp,ifp,iip] = diff / (len(ref_tts)-tlost)
                        durdiffs[imr,imt,ipp,ifp,iip] = ddiff / (len(ref_tts)-tlost)
                    else:
                        timediffs[imr,imt,ipp,ifp,iip] = 1000.
                        durdiffs[imr,imt,ipp,ifp,iip] = 1000.                        
                    losttransits[imr,imt,ipp,ifp,iip] = tlost        
                    
                
                

In [73]:
print(ref_tts)
tt = tts_all[mratset[0]][mtotset[0]][str(ppset[40])[:6]][str(fpset[3])[:6]][str(ipset[20])[:6]]
print(tt)
print(ref_tds)
td = tds_all[mratset[0]][mtotset[0]][str(ppset[40])[:6]][str(fpset[3])[:6]][str(ipset[20])[:6]]
print(td)
print(timediffs[0].shape)
#print(ref_tts-tt)
print(overlap[0,0,40,3,20])
print(timediffs[0,0,40,:,20])
#losttransits[10,3,49,:]
#durdiffs[10,3,49,:]
#np.max(overlap)
print(np.diff(ref_tts))

[ 2455332.87041014  2455460.2174625   2455586.42187743  2455712.356026
  2455838.3220782   2455964.64963706  2456092.50592814]
[ 2455072.15072068  2455198.03945609  2455324.05726218  2455450.30897101
  2455577.07292279  2455705.57034027  2455834.79774437  2455961.77398611
  2456088.10871223  2456214.17883335  2456340.1109963 ]
[ 0.48850902  0.23723435  0.181051    0.17016807  0.18964684  0.25943775
  0.54551647]
[ 0.20330538  0.22945058  0.27425698  0.35990815  0.57340739  1.38966328
  0.66839148  0.38132619  0.27927115  0.22882231  0.20048077]
(1, 101, 11, 41)
0.0
[ 6.09287326  6.23622742  6.37958158  6.52192579  6.66242325  6.80081426
  6.93695454  7.07133464  7.20432967  7.33657445  7.4685884 ]
[ 127.34705236  126.20441494  125.93414857  125.96605219  126.32755886
  127.85629109]


In [80]:
mratidx = 0
mtotidx = 0

timediff_minim = np.zeros([len(ppset),len(ipset)])
overlap_maxim = np.zeros([len(ppset),len(ipset)])
for pps in range(len(ppset)):
    for ips in range(len(ipset)):
        timediff_minim[pps,ips] = np.min(timediffs[mratidx,mtotidx,pps,:,ips])
        overlap_maxim[pps,ips] = np.max(overlap[mratidx,mtotidx,pps,:,ips])

timediff_minim[timediff_minim>500] = 20.
timediffs[timediffs>500] = 20
#plots
p.figure()
palette = p.cm.viridis
palette.set_over('grey', 500.0)
p.imshow(timediffs[mratidx,mtotidx,:,3,:],origin='lower',
         extent=[np.min(ipset),np.max(ipset),np.min(ppset),np.max(ppset)],
         aspect='auto',cmap=palette)
cbar = p.colorbar()
cbar.set_label('Time Difference (avg per transit, days)', rotation=270, labelpad=10)
p.ylabel('Planet_period')
p.xlabel('ip')
p.plot(ip0,p_p0,'rx')

p.figure()
palette = p.cm.viridis
p.imshow(overlap_maxim,origin='lower',
         extent=[np.min(ipset),np.max(ipset),np.min(ppset),np.max(ppset)],
         aspect='auto',cmap=palette)
cbar = p.colorbar()
cbar.set_label('Overlap with real transits (days)', rotation=270, labelpad=10)
p.ylabel('Planet_period')
p.xlabel('ip')
p.plot(ip0,p_p0,'rx')


IndexError: index 40 is out of bounds for axis 1 with size 1